<a href="https://colab.research.google.com/github/daikumatan/finetuning-hands-on/blob/main/Study_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tuning-competition

- https://llm-jp.github.io/tuning-competition/feature.html
- [llm-jp-3-1.8b](https://huggingface.co/llm-jp/llm-jp-3-1.8b)

## llm-jp-3-1.8bで動作確認してみる

In [1]:
pip install transformers datasets accelerate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# GPUが利用可能か確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# モデル名
model_name = "llm-jp/llm-jp-3-1.8b"

# トークナイザーとモデルをロードし、GPUに移動
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [3]:
# 入力テキスト
input_text = "こんにちは、私はAIです。今日の天気は"

# トークン化
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)

# モデルを使ったテキスト生成
outputs = model.generate(inputs, max_length=256, num_return_sequences=1, do_sample=True)

# 結果をデコード
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


こんにちは、私はAIです。今日の天気は雨のち晴れ。
雨の降り始める時間も予想されますね。
この前、雨の日の旅行は車酔いが心配だから
やめとこーっと思ってたけれどやっぱり雨だし、
晴れ女(私が)もさすがに雨か・・・という不安。
でも、雨降って地固まる(雨降って道路とかきれいになる)という
ことわざもあるし、晴れたらまた行こうと思います。
今日はこんな言葉を。
「『愛』は愛されることを望まず、自分の気持ちをまず第一に考える事が
『幸せ』になる上では肝心です。
愛することとは、あなたの愛する人のために『自分の気持ち』を
『犠牲』にする事ではありません。」
自分も相手も『幸せ』になるのが『愛』だと思っているけれど、
幸せにするのは自分じゃなく人。
自分の気持ちばかりを優先すると自分の『幸せ』すら
得られなくなってしまうこともあるよね。
人を『愛す』ことが『自己犠牲』になってしまってもいいものか。
そんな風に思う事も時にはあるけれど
それは自分を愛せるようになるまで少し時間がかかってのこと。
まずは自分が『愛』に満たされること。
それが自分を『幸せ』にしていくんじゃないのかなー。

Tweet


## FTのHello World

In [4]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

# GPUデバイスの確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# モデル名
model_name = "llm-jp/llm-jp-3-1.8b"

# トークナイザーとモデルをロードし、GPUに移動
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Using device: cuda


### 利用可能な日本語テストデータセットの確認

In [5]:
from datasets import get_dataset_config_names

configs = get_dataset_config_names("graelo/wikipedia")
ja_configs = [config for config in configs if config.endswith(".ja")]
print(ja_configs)

README.md:   0%|          | 0.00/201k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/4.81k [00:00<?, ?B/s]

lang_def.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

The repository for graelo/wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/graelo/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
['20230901.ja', '20230601.ja']


In [6]:
# データセットの準備（正しい構成名を使用）
# Hugging Face公式ページで確認した最新の日本語Wikipediaダンプを使用
ja_data = "20230901.ja"
dataset = load_dataset("graelo/wikipedia", ja_data, split="train[:1%]")  # データセットの1%を使用
print(f"Loaded dataset: {dataset}")

data/20230901/ja/info.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

train-0001-of-0016.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

train-0002-of-0016.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

train-0003-of-0016.parquet:   0%|          | 0.00/250M [00:00<?, ?B/s]

train-0004-of-0016.parquet:   0%|          | 0.00/251M [00:00<?, ?B/s]

...

train-0014-of-0016.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

train-0015-of-0016.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

train-0016-of-0016.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1383531 [00:00<?, ? examples/s]

Loaded dataset: Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 13835
})


### 試しにロードしたデータを確認

In [7]:
import json
# データセットの最初の数行を表示
print(f"Number of samples: {len(dataset)}")
print("Example data:")
for i in range(3):
    print(f"Sample {i}:")
    print(json.dumps(dataset[i], ensure_ascii=False, indent=2))
    print("-" * 50)  # 区切り線

Number of samples: 13835
Example data:
Sample 0:
{
  "id": "5",
  "url": "https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%B3%E3%83%91%E3%82%B5%E3%83%B3%E3%83%89",
  "title": "アンパサンド",
  "text": "アンパサンド（&, ）は、並立助詞「…と…」を意味する記号である。ラテン語で「…と…」を表す接続詞 \"et\" の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、\"et\" の合字であることが容易にわかる字形を使用している。\n\n語源 \n\n英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字（\"A\", \"I\", かつては \"O\" も）については、伝統的にラテン語の per se（それ自体）を用いて \"A per se A\" のように唱えられていた。また、アルファベットの最後に、27番目の文字のように \"&\" を加えることも広く行われていた。\"&\" はラテン語で et と読まれていたが、のちに英語で and と読まれるようになった。結果として、アルファベットの復唱の最後は \"X, Y, Z, and per se and\" という形になった。この最後のフレーズが繰り返されるうちに \"ampersand\" となまっていき、この言葉は1837年までには英語の一般的な語法となった。\n\nアンドレ＝マリ・アンペールがこの記号を自身の著作で使い、これが広く読まれたため、この記号が \"Ampère's and\" と呼ばれるようになったという誤った語源俗説がある。\n\n歴史 \n\nアンパサンドの起源は1世紀の古ローマ筆記体にまでさかのぼることができる。古ローマ筆記体では、E と T はしばしば合字として繋げて書かれていた（左図「アンパサンドの変遷」の字形1）。それに続く、流麗さを増した新ローマ筆記体では、さまざまな合字が極めて頻繁に使われるようになった。字形2と3は4世紀中頃における et の合字の例である。その後、9世紀のカロリング小文字体に至るラテン文字の変遷の過程で、合字の使用は

### テストデータセットのトークン化

In [8]:
# トークナイズ処理
def tokenize_function(examples):
    # 'text' 列をトークナイズし、labels を input_ids と同じに設定
    tokens = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )
    tokens["labels"] = tokens["input_ids"].copy()  # labels を input_ids と同じにする
    return tokens

# データセットのトークナイズ
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["id", "url", "title", "text"]  # 不要な列を削除
)

Map:   0%|          | 0/13835 [00:00<?, ? examples/s]

In [9]:
# データセットを分割
train_size = int(0.8 * len(tokenized_dataset))
train_dataset = tokenized_dataset.select(range(train_size))
eval_dataset = tokenized_dataset.select(range(train_size, len(tokenized_dataset)))

### データの形状確認してみよう

In [21]:
print(train_dataset[0].keys())
print(len(train_dataset[0]["input_ids"]))
print(len(train_dataset[0]["token_type_ids"]))
print(len(train_dataset[0]["attention_mask"]))
print(len(train_dataset[0]["labels"]))
print(train_dataset[0]["input_ids"][:3])
print(train_dataset[0]["token_type_ids"][:3])
print(train_dataset[0]["attention_mask"][:3])
print(train_dataset[0]["labels"][:3])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
128
128
128
128
[1, 41284, 29893]
[0, 0, 0]
[1, 1, 1]
[1, 41284, 29893]


## Fine Tuningの実行

In [22]:
# import os
# os.environ["WANDB_DISABLED"] = "true"
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# トレーニング設定
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    #gradient_accumulation_steps=4,  # 勾配を4ステップ分累積
    num_train_epochs=1,
    weight_decay=0.01,
    save_steps=1000,
    save_total_limit=2,
    logging_dir="./logs",
    remove_unused_columns=False,  # 未使用列を削除しない
    report_to=[],  # wandb無効化
    fp16=True,  # 混合精度トレーニング
    optim="adamw_torch",  # メモリ効率が高いオプティマイザー
)

# Trainerの設定
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

<ipython-input-22-310231733740>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
# 学習開始
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.262600,2.298065


TrainOutput(global_step=1384, training_loss=2.26766517947864, metrics={'train_runtime': 528.7056, 'train_samples_per_second': 20.934, 'train_steps_per_second': 2.618, 'total_flos': 1.4141535157223424e+16, 'train_loss': 2.26766517947864, 'epoch': 1.0})

In [24]:
trainer.save_model("/content/drive/MyDrive/StudyFineTuning")


In [27]:
# テキスト生成用コード
test_text = "アンパサンドとは何ですか？"
inputs = tokenizer(test_text, return_tensors="pt").to("cuda")

# token_type_ids を削除
if "token_type_ids" in inputs:
    del inputs["token_type_ids"]

# 生成
outputs = model.generate(**inputs, max_length=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


アンパサンドとは何ですか？

アンパサンド（）は、記号の一つで、記号の前に置かれる文字の前に置かれる文字である。

記号の前に置かれる文字 

記号の前に置かれる文字は、記号の前に置かれる文字である。記号の前に置かれる文字は、記号の前に置かれる文字である。記号の前に置かれる文字は、記号の前に置かれる文字である。記号の前に置かれる文字は、記号の前に置かれる文字である。記号の前に置かれる文字は、記号の前に置かれる文字である。記号の前に置かれる文字は、記号の前に置かれる文字である。記号の
